In [9]:
import json
import os
import time
import requests
import threading
import logging
import logging.handlers
import colorlog
from halo import Halo
from datetime import datetime, timedelta
from dotenv import load_dotenv
from web3 import Web3
from py_clob_client.client import ClobClient
from py_clob_client.clob_types import MarketOrderArgs, OrderType
from py_clob_client.order_builder.constants import BUY, SELL
from typing import Dict, List, Tuple, Optional, Any, Union
from collections import deque, defaultdict
from threading import Lock, Event
from concurrent.futures import ThreadPoolExecutor
import signal
import sys
from dataclasses import dataclass
from enum import Enum
import functools
from queue import Queue

In [10]:
load_dotenv(".env")
TRADE_UNIT = float(os.getenv("trade_unit"))
SLIPPAGE_TOLERANCE = float(os.getenv("slippage_tolerance"))
PCT_PROFIT = float(os.getenv("pct_profit"))
PCT_LOSS = float(os.getenv("pct_loss"))
CASH_PROFIT = float(os.getenv("cash_profit"))
CASH_LOSS = float(os.getenv("cash_loss"))
SPIKE_THRESHOLD = float(os.getenv("spike_threshold"))
SOLD_POSITION_TIME = float(os.getenv("sold_position_time"))
HOLDING_TIME_LIMIT = float(os.getenv("holding_time_limit"))
PRICE_HISTORY_SIZE = int(os.getenv("price_history_size"))
COOLDOWN_PERIOD = int(os.getenv("cooldown_period"))
KEEP_MIN_SHARES = int(os.getenv("keep_min_shares"))
MAX_CONCURRENT_TRADES = int(os.getenv("max_concurrent_trades"))
MIN_LIQUIDITY_REQUIREMENT = float(os.getenv("min_liquidity_requirement"))
# Web3 and API setup
WEB3_PROVIDER = "https://polygon-rpc.com"
YOUR_PROXY_WALLET = Web3.to_checksum_address(os.getenv("YOUR_PROXY_WALLET"))
BOT_TRADER_ADDRESS = Web3.to_checksum_address(os.getenv("BOT_TRADER_ADDRESS"))
USDC_CONTRACT_ADDRESS = os.getenv("USDC_CONTRACT_ADDRESS")
POLYMARKET_SETTLEMENT_CONTRACT = os.getenv("POLYMARKET_SETTLEMENT_CONTRACT")
PRIVATE_KEY = os.getenv("PK")

In [11]:
# Constants
MAX_RETRIES = 3                 # Number of retries for API calls
BASE_DELAY = 1                  # Base delay for retries
MAX_ERRORS = 5                  # Maximum number of errors before shutting down
API_TIMEOUT = 10                # Timeout for API requests
REFRESH_INTERVAL = 3600         # Refresh interval for API credentials
COOLDOWN_PERIOD = 30            # Cooldown period for trades
THREAD_POOL_SIZE = 3            # Number of threads in the thread pool 
MAX_QUEUE_SIZE = 1000           # Maximum number of items in the queue
THREAD_CHECK_INTERVAL = 5       # Interval for checking thread status
THREAD_RESTART_DELAY = 2        # Delay before restarting a thread

In [12]:
web3 = Web3(Web3.HTTPProvider(WEB3_PROVIDER))

In [13]:
def initialize_clob_client(max_retries: int = 3) -> ClobClient:
    for attempt in range(max_retries):
        try:
            client = ClobClient(
                host="https://clob.polymarket.com",
                key=PRIVATE_KEY,
                chain_id=137,
                signature_type=2,
                funder=YOUR_PROXY_WALLET
            )
            api_creds = client.create_or_derive_api_creds()
            client.set_api_creds(api_creds)
            return client
        except Exception as e:
            if attempt == max_retries - 1:
                raise
            logger.warning(f"Failed to initialize ClobClient (attempt {attempt + 1}/{max_retries}): {e}")
            time.sleep(2 ** attempt)
    raise RuntimeError("Failed to initialize ClobClient after maximum retries")

client = initialize_clob_client()

In [21]:
def ensure_usdc_allowance(required_amount: float) -> bool:
    """Ensure USDC allowance with proper error handling"""
    max_retries = MAX_RETRIES
    base_delay = BASE_DELAY
    
    for attempt in range(max_retries):
        try:
            contract = web3.eth.contract(address=USDC_CONTRACT_ADDRESS, abi=[
                {"constant": True, "inputs": [{"name": "owner", "type": "address"}, {"name": "spender", "type": "address"}],
                 "name": "allowance", "outputs": [{"name": "", "type": "uint256"}],
                 "payable": False, "stateMutability": "view", "type": "function"},
                {"constant": False, "inputs": [{"name": "spender", "type": "address"}, {"name": "value", "type": "uint256"}],
                 "name": "approve", "outputs": [{"name": "", "type": "bool"}],
                 "payable": False, "stateMutability": "nonpayable", "type": "function"}
            ])

            current_allowance = contract.functions.allowance(BOT_TRADER_ADDRESS , POLYMARKET_SETTLEMENT_CONTRACT).call()
            required_amount_with_buffer = int(required_amount * 1.1 * 10**6)
            
            print(current_allowance)
            
            if current_allowance >= required_amount_with_buffer:
                return True

            
            new_allowance = max(current_allowance, required_amount_with_buffer)
            txn = contract.functions.approve(POLYMARKET_SETTLEMENT_CONTRACT, new_allowance).build_transaction({
                "from": BOT_TRADER_ADDRESS,
                "gas": 200000,
                "gasPrice": web3.eth.gas_price,
                "nonce": web3.eth.get_transaction_count(BOT_TRADER_ADDRESS),
                "chainId": 137
            })
            
            signed_txn = web3.eth.account.sign_transaction(txn, private_key=PRIVATE_KEY)
            tx_hash = web3.eth.send_raw_transaction(signed_txn.raw_transaction)
            receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
            
            if receipt.status == 1:
                return True
            else:
                raise TradingError(f"USDC allowance update failed: {tx_hash.hex()}")
                
        except Exception as e:
            if attempt == max_retries - 1:
                raise TradingError(f"Failed to update USDC allowance: {e}")
            time.sleep(base_delay * (2 ** attempt))
    
    return False

In [23]:
class BotError(Exception):
    pass

class ConfigurationError(BotError):
    pass

class NetworkError(BotError):
    pass

class TradingError(BotError):
    pass

class ValidationError(BotError):
    pass

# Custom Types
@dataclass
class TradeInfo:
    entry_price: float
    entry_time: float
    amount: float
    bot_triggered: bool

@dataclass
class PositionInfo:
    eventslug: str
    outcome: str
    asset: str
    avg_price: float
    shares: float
    current_price: float
    initial_value: float
    current_value: float
    pnl: float
    percent_pnl: float
    realized_pnl: float

class TradeType(Enum):
    BUY = "buy"
    SELL = "sell"

In [24]:
def setup_logging() -> logging.Logger:
    """Setup enhanced logging configuration with both file and console handlers"""
    # Create logs directory if it doesn't exist
    os.makedirs('logs', exist_ok=True)
    
    # Create a logger
    logger = logging.getLogger('polymarket_bot')
    logger.setLevel(logging.INFO)
    
    # Clear any existing handlers
    logger.handlers = []
    
    # Create formatters
    file_formatter = logging.Formatter(
        '%(asctime)s | %(levelname)-8s | %(threadName)-12s | %(name)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )
    
    console_formatter = colorlog.ColoredFormatter(
        '%(log_color)s%(asctime)s | %(levelname)-8s | %(threadName)-12s | %(name)s | %(message)s%(reset)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        log_colors={
            'DEBUG': 'cyan',
            'INFO': 'green',
            'WARNING': 'yellow',
            'ERROR': 'red',
            'CRITICAL': 'red,bg_white'
        }
    )
    
    # File handler - Rotating file handler with size limit
    file_handler = logging.handlers.RotatingFileHandler(
        'logs/polymarket_bot.log',
        maxBytes=10*1024*1024,  # 10MB
        backupCount=5,
        encoding='utf-8'
    )
    file_handler.setFormatter(file_formatter)
    file_handler.setLevel(logging.INFO)
    
    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setFormatter(console_formatter)
    console_handler.setLevel(logging.INFO)
    
    # Add handlers to logger
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)
    
    return logger

# Initialize logger
logger = setup_logging()

In [27]:
def fetch_positions_with_retry(max_retries: int = MAX_RETRIES) -> Dict[str, List[PositionInfo]]:
    for attempt in range(max_retries):
        try:
            url = f"https://data-api.polymarket.com/positions?user={YOUR_PROXY_WALLET}"
            logger.info(f"🔄 Fetching positions from {url} (attempt {attempt + 1}/{max_retries})")
            
            response = requests.get(url, timeout=API_TIMEOUT)
            logger.info(f"📡 API Response Status: {response.status_code}")
            
            if response.status_code != 200:
                logger.error(f"❌ API Error: {response.status_code} - {response.text}")
                raise NetworkError(f"API returned status code {response.status_code}")
            
            response.raise_for_status()
            data = response.json()
            
            logger.info(f"response data: {data}")
            
            if not isinstance(data, list):
                logger.error(f"❌ Invalid response format: {type(data)}")
                logger.error(f"Response content: {data}")
                raise ValidationError(f"Invalid response format from API: {type(data)}")
            
            if not data:
                logger.warning("⚠️ No positions found in API response. Waiting for positions...")
                return {}
                
            positions: Dict[str, List[PositionInfo]] = {}
            for pos in data:
                event_id = pos.get("conditionId") or pos.get("eventId") or pos.get("marketId")
                if not event_id:
                    logger.warning(f"⚠️ Skipping position with no event ID: {pos}")
                    continue
                    
                if event_id not in positions:
                    positions[event_id] = []
                    
                try:
                    position_info = PositionInfo(
                        eventslug=pos.get("eventSlug", ""),
                        outcome=pos.get("outcome", ""),
                        asset=pos.get("asset", ""),
                        avg_price=float(pos.get("avgPrice", 0)),
                        shares=float(pos.get("size", 0)),
                        current_price=float(pos.get("curPrice", 0)),
                        initial_value=float(pos.get("initialValue", 0)),
                        current_value=float(pos.get("currentValue", 0)),
                        pnl=float(pos.get("cashPnl", 0)),
                        percent_pnl=float(pos.get("percentPnl", 0)),
                        realized_pnl=float(pos.get("realizedPnl", 0))
                    )
                    positions[event_id].append(position_info)
                    logger.debug(f"✅ Added position: {position_info}")
                except (ValueError, TypeError) as e:
                    logger.error(f"❌ Error parsing position data: {e}")
                    logger.error(f"Problematic position data: {pos}")
                    continue
            
            logger.info(f"✅ Successfully fetched {len(positions)} positions")
            return positions
            
        except requests.RequestException as e:
            logger.error(f"❌ Network error in fetch_positions (attempt {attempt + 1}/{max_retries}): {str(e)}")
            if attempt == max_retries - 1:
                raise NetworkError(f"Failed to fetch positions after {max_retries} attempts: {e}")
            time.sleep(2 ** attempt)
        except (ValueError, ValidationError) as e:
            logger.error(f"❌ Validation error in fetch_positions (attempt {attempt + 1}/{max_retries}): {str(e)}")
            if attempt == max_retries - 1:
                raise ValidationError(f"Invalid data received from API: {e}")
            time.sleep(2 ** attempt)
        except Exception as e:
            logger.error(f"❌ Unexpected error in fetch_positions (attempt {attempt + 1}/{max_retries}): {str(e)}")
            if attempt == max_retries - 1:
                raise NetworkError(f"Failed to fetch positions after {max_retries} attempts: {e}")
            time.sleep(2 ** attempt)
    
    raise NetworkError("Failed to fetch positions after maximum retries")

In [28]:
p = fetch_positions_with_retry(5)

2025-10-29 21:43:44 | INFO     | MainThread   | polymarket_bot | 🔄 Fetching positions from https://data-api.polymarket.com/positions?user=0xDf089e4A6A588C49eFcADEBf6Db7612b8F029a18 (attempt 1/5)
2025-10-29 21:43:45 | INFO     | MainThread   | polymarket_bot | 📡 API Response Status: 200
2025-10-29 21:43:45 | INFO     | MainThread   | polymarket_bot | response data: []
2025-10-29 21:43:45 | WARNING  | MainThread   | polymarket_bot | ⚠️ No positions found in API response. Waiting for positions...


In [ ]:
def has_orderbook(client, token_id: str) -> bool:
    try:
        ob = client.get_order_book(token_id)
        return bool((getattr(ob, "bids", None) and len(ob.bids) > 0) or
                    (getattr(ob, "asks", None) and len(ob.asks) > 0))
    except Exception:
        # 包括 PolyApiException 404 在内，统统视为无订单簿
        return False

# 拉市场并筛选
markets = client.get_simplified_markets().get("data", [])
print(f"markets len is: {len(markets)}")
pairs = []
for m in markets[:50]:  # 可根据需要调整数量
    tokens = m.get("tokens", [])
    if isinstance(tokens, list) and len(tokens) >= 2:
        a0 = str(tokens[0].get("token_id", ""))
        a1 = str(tokens[1].get("token_id", ""))
        if a0 and a1 and (has_orderbook(client, a0) or has_orderbook(client, a1)):
            pairs.append((a0, a1))

print(f"筛到 {len(pairs)} 个有订单簿的资产对（至少一侧有）")
print(pairs[:5])  # 看看前几个

# buy_price = client.get_price(id, side='buy')

PolyApiException: PolyApiException[status_code=None, error_message=Request exception!]

In [54]:
orderbook = client.get_order_book(
        "33945469250963963541781051637999677727672635213493648594066577298999471399137"
)
print("orderbook", orderbook)

orderbook OrderBookSummary(market='0xebddfcf7b4401dade8b4031770a1ab942b01854f3bed453d5df9425cd9f211a9', asset_id='33945469250963963541781051637999677727672635213493648594066577298999471399137', timestamp='1761749559265', bids=[OrderSummary(price='0.001', size='709189.46'), OrderSummary(price='0.002', size='1000010'), OrderSummary(price='0.003', size='334342'), OrderSummary(price='0.004', size='500100'), OrderSummary(price='0.005', size='101000'), OrderSummary(price='0.006', size='300010'), OrderSummary(price='0.007', size='520000'), OrderSummary(price='0.008', size='1000'), OrderSummary(price='0.01', size='1722.49'), OrderSummary(price='0.011', size='20000'), OrderSummary(price='0.013', size='20000'), OrderSummary(price='0.014', size='30000'), OrderSummary(price='0.015', size='10000'), OrderSummary(price='0.017', size='20000'), OrderSummary(price='0.018', size='60000'), OrderSummary(price='0.02', size='4500'), OrderSummary(price='0.021', size='50500'), OrderSummary(price='0.025', size=

In [48]:
import requests
import json

EVENT_SLUG = "new-york-city-mayoral-election"
url = f"https://gamma-api.polymarket.com/events/slug/{EVENT_SLUG}" 
print(f"-> 正在查询 Event 详情: {url}")

response = requests.get(url)

if response.status_code == 200:
    event_data = response.json()
    
    # 验证返回的数据是否与 Slug 匹配
    if event_data.get('slug') != EVENT_SLUG:
        print(f"❌ 警告：返回的 Slug ({event_data.get('slug')}) 与请求的 Slug 不匹配。")
        
    # **关键步骤：提取内嵌的市场列表**
    markets = event_data.get('markets', [])

    print(f"✅ 成功找到 Event ID: {event_data.get('id')}")
    print(f"✅ 包含 {len(markets)} 个市场。")
    
    # 示例：遍历并打印每个市场的问题
    for market in markets:
        print(f"   - Market ID: {market.get('id')}, Question: {market.get('question')}")
        
    # 现在您可以从 markets 列表中的特定 market 提取 asset_id
    # asset_id = markets[0]['tokens'][0]['asset_id'] 

else:
    print(f"❌ 错误：调用 Gamma API 失败: {response.status_code} {response.reason}")


-> 正在查询 Event 详情: https://gamma-api.polymarket.com/events/slug/new-york-city-mayoral-election
✅ 成功找到 Event ID: 23246
✅ 包含 19 个市场。
   - Market ID: 538928, Question: Will Eric Adams win the 2025 NYC mayoral election?
   - Market ID: 538929, Question: Will Andrew Cuomo win the 2025 NYC mayoral election?
   - Market ID: 538930, Question: Will Curtis Sliwa win the 2025 NYC mayoral election?
   - Market ID: 538931, Question: Will Jim Walden win the 2025 NYC mayoral election?
   - Market ID: 538932, Question: Will Zohran Mamdani win the 2025 NYC mayoral election?
   - Market ID: 538939, Question: Will Michael Bloomberg win the 2025 NYC mayoral election?
   - Market ID: 538942, Question: Will Person D win the 2025 NYC mayoral election?
   - Market ID: 538933, Question: Will Zellnor Myrie win the 2025 NYC mayoral election?
   - Market ID: 538937, Question: Will Adrienne Adams win the 2025 NYC mayoral election?
   - Market ID: 538934, Question: Will Scott Stringer win the 2025 NYC mayoral electi

In [52]:
MARKET_ID = "538932" 
url = f"https://gamma-api.polymarket.com/markets/{MARKET_ID}" 
print(f"-> 正在查询 Market 详情: {url}")

response = requests.get(url)
print(response.json())
if response.status_code == 200:
    market_details = response.json()
    
    # 打印部分数据结构（可选）
    # print(json.dumps(market_details, indent=4))
    
    # --- 关键步骤：提取 Token ID ---
    
    # 市场详细信息中的 'tokens' 字段是一个包含 YES/NO 结果的列表
    tokens_list = market_details.get('tokens', [])
    print(tokens_list)
    if tokens_list and len(tokens_list) == 2:
        # Polymarket 的二元市场通常包含两个 Token (YES/NO)
        
        # 结果 1 (通常是 YES token)
        yes_token = tokens_list[0]
        yes_asset_id = yes_token.get('asset_id')
        yes_label = yes_token.get('label')

        # 结果 2 (通常是 NO token)
        no_token = tokens_list[1]
        no_asset_id = no_token.get('asset_id')
        no_label = no_token.get('label')
        
        # 结果展示
        print("\n✅ 成功获取 Token ID:")
        print(f"   - 市场问题: {market_details.get('question')}")
        print(f"   - {yes_label} Asset ID: **{yes_asset_id}**")
        print(f"   - {no_label} Asset ID: **{no_asset_id}**")

    else:
        print("❌ 警告：市场数据中未找到有效的 Token 列表。")

else:
    print(f"❌ 错误：调用 Market API 失败: {response.status_code} {response.reason}")

-> 正在查询 Market 详情: https://gamma-api.polymarket.com/markets/538932
{'id': '538932', 'question': 'Will Zohran Mamdani win the 2025 NYC mayoral election?', 'conditionId': '0xebddfcf7b4401dade8b4031770a1ab942b01854f3bed453d5df9425cd9f211a9', 'slug': 'will-zohran-mamdani-win-the-2025-nyc-mayoral-election', 'resolutionSource': '', 'endDate': '2025-11-04T12:00:00Z', 'liquidity': '581401.89335', 'startDate': '2025-04-22T16:10:05.987Z', 'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/will-zohran-mamdani-win-the-2025-nyc-mayoral-election-EscSJQTT6hWg.jpg', 'icon': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/will-zohran-mamdani-win-the-2025-nyc-mayoral-election-EscSJQTT6hWg.jpg', 'description': 'The 2025 New York City mayoral election will be held on November 4, 2025, to elect the mayor of New York City.\n\nThis market will resolve according to the candidate that wins the election.\n\nThe primary resolution source for this market will be a consensus of credible reporting

In [57]:
p = client.get_price("33945469250963963541781051637999677727672635213493648594066577298999471399137", side="BUY")
print(p)

{'price': '0.882'}


In [ ]:
BASE_URL = "https://gamma-api.polymarket.com/events"
all_event_slugs = []
next_cursor = "" # 用于分页的游标，初始为空

print("-> 正在从 Polymarket API 获取所有有效的 Event Slug...")

while True:
    # 构造请求 URL 和参数
    params = {
        "closed": "false",  # 仅获取未关闭的事件
        "archived": "false", # 排除已归档事件 (可选)
        "limit": 100,      # 每页限制 100 个结果
    }
    if next_cursor:
        params["next_cursor"] = next_cursor
        
    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()  # 检查是否为 200 状态码
        
        data = response.json()
        print(data)
        print(type(data))
        events = data[0].get('data', [])
        
        # 提取当前页的所有 Event Slug
        current_slugs = [event.get('slug') for event in events if event.get('slug')]
        all_event_slugs.extend(current_slugs)
        
        print(f"   - 已获取 {len(all_event_slugs)} 个 Event Slug...")
        
        # 检查是否还有下一页
        next_cursor = data.get('next_cursor')
        if not next_cursor:
            break  # 没有下一页，退出循环
            
        # 遵守 API 速率限制，等待 1 秒后再进行下一页请求
        time.sleep(1) 
        
    except requests.exceptions.RequestException as e:
        print(f"❌ API 请求出错: {e}")
        break

print(f"\n✅ 任务完成！总共获取到 {len(all_event_slugs)} 个有效的 Event Slug。")

# 打印前 10 个示例
print("\n--- 前 10 个 Event Slug 示例 ---")
for slug in all_event_slugs[:10]:
    print(f"- {slug}")

-> 正在从 Polymarket API 获取所有有效的 Event Slug...
[{'id': '16084', 'ticker': 'fed-rate-hike-in-2025', 'slug': 'fed-rate-hike-in-2025', 'title': 'Fed rate hike in 2025?', 'description': 'This market will resolve to “Yes” if the upper bound of the target federal funds rate is increased at any point between January 1, 2025 and the Fed\'s December 2025 meeting, currently scheduled for December 9-10. Otherwise, this market will resolve to “No”.\n\nThis market may not resolve to "No" until the Fed has released its rate changes information following its December meeting.\n\nThe primary resolution source for this market will be the official website of the Federal Reserve (https://www.federalreserve.gov/monetarypolicy/openmarket.htm), however a consensus of credible reporting may also be used.', 'startDate': '2024-12-29T22:50:44.013518Z', 'creationDate': '2024-12-29T22:50:44.013516Z', 'endDate': '2025-12-10T12:00:00Z', 'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/will-the-fed-raise-

AttributeError: 'list' object has no attribute 'get'